# 随机梯度下降法

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
m = 100000
x = np.random.normal(size = m)
X = x.reshape(-1,1)
y = 4 * x + 3.0 + np.random.normal(0,3, size = m)

In [3]:
def J(theta, X_b, y):
    try: 
        return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
    except:
        return float('inf')
def dJ(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2 / len(y)

def gradient_descent(X_b, y , initial_theta, eta, n_iters=1e6, epsilon= 1e-8):
    theta = initial_theta
    cur_iter = 0
    
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        cur_iter += 1
    return theta

In [4]:
%%time
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.001
theta = gradient_descent(X_b, y, initial_theta, eta)

CPU times: user 5.67 s, sys: 201 ms, total: 5.87 s
Wall time: 6.14 s


In [5]:
theta

array([2.98645553, 3.99884456])

In [6]:
def dJ_sgd(theta, X_b_i, y_i):
    return X_b_i.T.dot(X_b_i.dot(theta) - y_i) * 2

In [7]:
def sgd(X_b, y, initial_theta, n_iters):
    
    t0 = 5
    t1 = 50
    def learning_rate(t):
        return t0 / (t + t1)
    theta = initial_theta
    for cur_iter in range(n_iters):
        rand_i = np.random.randint(len(X_b))
        gradient = dJ_sgd(theta, X_b[rand_i], y[rand_i])
        theta = theta - learning_rate(cur_iter) * gradient
        
    return theta

In [8]:
%%time
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b, y, initial_theta, n_iters=len(X_b)//3)

CPU times: user 241 ms, sys: 4.23 ms, total: 245 ms
Wall time: 251 ms


In [9]:
theta

array([3.01876644, 3.97149275])

In [10]:
from kNN.LinearRegression import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit_sgd(X, y, n_iters = 2)

LinearRegression()

In [13]:
lin_reg.coef_

array([3.99623939])

In [12]:
lin_reg.intercept_

2.9812379018707724

In [14]:
from sklearn import datasets

In [15]:
boston = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [16]:
from kNN.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, seed = 666)

In [19]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(X_train)
X_train_standard = standardScaler.transform(X_train)
X_test_standard = standardScaler.transform(X_test)

In [20]:
from kNN.LinearRegression import LinearRegression

lin_reg = LinearRegression()
%time lin_reg.fit_sgd(X_train_standard, y_train, n_iters=2)
lin_reg.score(X_test_standard, y_test)

CPU times: user 7.79 ms, sys: 2.72 ms, total: 10.5 ms
Wall time: 8.23 ms


0.7865171620468298

In [22]:
%time lin_reg.fit_sgd(X_train_standard, y_train, n_iters=500)
lin_reg.score(X_test_standard, y_test)

CPU times: user 1.16 s, sys: 11.9 ms, total: 1.17 s
Wall time: 1.41 s


0.8129737678710172

# scikit-learn中的SGD

In [23]:
from sklearn.linear_model import SGDRegressor

In [24]:
sgd_reg = SGDRegressor()
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard, y_test)

CPU times: user 2.98 ms, sys: 5.13 ms, total: 8.11 ms
Wall time: 9.89 ms


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.8007994916924736

In [26]:
sgd_reg = SGDRegressor(n_iter=500)
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard, y_test)

CPU times: user 26.4 ms, sys: 1.8 ms, total: 28.2 ms
Wall time: 29 ms


0.812390491407812

# 调试梯度

In [27]:
import numpy as np
import matplotlib.pyplot as plt

In [29]:
np.random.seed(666)
X = np.random.random(size=(1000,10))

In [30]:
true_theta = np.arange(1, 12, dtype = float)

In [31]:
X_b = np.hstack([np.ones((len(X), 1)), X])
y = X_b.dot(true_theta) + np.random.normal(size = 1000)

In [32]:
X.shape

(1000, 10)

In [33]:
y.shape

(1000,)

In [34]:
true_theta

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [35]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(X_b)
    except:
        return float('inf')

In [36]:
def dJ_math(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)

In [37]:
def dJ_debug(theta, X_b, y, epsilon=0.01):
    res = np.empty(len(theta))
    for i in range(len(theta)):
        theta_1 = theta.copy()
        theta_1[i] += epsilon
        theta_2 = theta.copy()
        theta_2[i] -= epsilon
        res[i] = (J(theta_1, X_b, y) - J(theta_2, X_b, y)) / (2 * epsilon)
    return res

In [39]:
def gradient_descent(dJ, X_b, y, initial_theta, eta, n_iters = 1e4, epsilon=1e-8):
    theta = initial_theta
    cur_iter = 0
    
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        cur_iter += 1
    return theta

In [43]:
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
%time theta = gradient_descent(dJ_debug, X_b, y, initial_theta, eta)
theta

CPU times: user 6.8 s, sys: 1.78 s, total: 8.59 s
Wall time: 8.93 s


array([ 1.1251597 ,  2.05312521,  2.91522497,  4.11895968,  5.05002117,
        5.90494046,  6.97383745,  8.00088367,  8.86213468,  9.98608331,
       10.90529198])

In [45]:
%time theta = gradient_descent(dJ_math, X_b, y, initial_theta, eta)
theta

CPU times: user 883 ms, sys: 234 ms, total: 1.12 s
Wall time: 1.13 s


array([ 1.1251597 ,  2.05312521,  2.91522497,  4.11895968,  5.05002117,
        5.90494046,  6.97383745,  8.00088367,  8.86213468,  9.98608331,
       10.90529198])